In [1]:
import pyodbc
from meyerDB import cable_connection
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeavePOut
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
import sklearn.preprocessing as pp
from sklearn.model_selection import GridSearchCV
from copy import deepcopy

# init db connection
conn = pyodbc.connect(cable_connection)
cursor = conn.cursor()
print('Database connection ok')

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            try:
                value = str(round(100*field, 1)).replace('.', ',') + '%'
            except:
                value = field
            html += "<td><h4>%s</h4><td>"%(value)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

Database connection ok


In [2]:
# Whole ship cable quantities
# Predictors:
# GT
# Algorithms:
# Linear regression
# KNN regression
# Mean

# Get the data
cursor.execute("SELECT gross_tonnage, sum(amount) FROM routed as r"
    " LEFT JOIN projects as p ON p.project_id=r.project_id"
    " GROUP BY r.project_id, gross_tonnage")
data = np.array(cursor.fetchall()).astype('float32')
print('Data array: ', data.shape)
X = data[:, 0].reshape((-1, 1))
print('Features', X.shape)
y = data[:, -1]


# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

# init regression model
models = dict()
models['Linear regression'] = linear_model.LinearRegression()
models['KNN Regression'] = KNeighborsRegressor()
models['Mean'] = DummyRegressor(strategy='mean')

scaler = pp.MinMaxScaler()

# train and evaluate all models with lpo
table_rows = [['Model', 'mean', 'std', 'min', 'max']]

for model_name in models.keys():
    model = models[model_name]
    scores = []
    for train_i, test_i in lpo.split(X):
        if model_name == 'KNN Regression':
            clf = GridSearchCV(model, {'n_neighbors':[1, 3, 5], 'weights': ['uniform', 'distance']}, cv=3)
            X_train = deepcopy(scaler.fit_transform(X[train_i]))
            clf.fit(X_train, y[train_i], )
            X_test = deepcopy(scaler.transform(X[test_i])) #normalize features
            preds = clf.predict(X_test)
            scores.append(mape(y[test_i], preds))
        else:
            model.fit(X[train_i], y[train_i])
            preds = model.predict(X[test_i])
            scores.append(mape(y[test_i], preds))
    scores = np.array(scores)
    table_rows.append((model_name, scores.mean(), scores.std(), scores.min(), scores.max()))
    with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
        if model_name == 'Mean':
            line = '-;Whole ship cable quantity;m;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        else:
            line = 'GT;Whole ship cable quantity;m;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        file_object.write(line)
display_table(table_rows)


Data array:  (11, 2)
Features (11, 1)


In [3]:
# Whole ship cable quantities
# Predictors:
# Ship squares
# Algorithms:
# Linear regression
# KNN regression

# Get the data
cursor.execute("SELECT ship_squares , sum(amount) FROM routed as r"
    " LEFT JOIN (SELECT project_id, sum(squares) as ship_squares FROM areas GROUP BY project_id) as a ON a.project_id=r.project_id"
    " GROUP BY r.project_id, ship_squares")
data = np.array(cursor.fetchall()).astype('float32')
print('Data array: ', data.shape)
X = data[:, 0].reshape((-1, 1))
print('Features', X.shape)
y = data[:, -1]


# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

# init regression model
models = dict()
models['Linear regression'] = linear_model.LinearRegression()
models['KNN Regression'] = KNeighborsRegressor()

scaler = pp.MinMaxScaler()

# train and evaluate all models with lpo
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
for model_name in models.keys():
    model = models[model_name]
    scores = []
    for train_i, test_i in lpo.split(X):
        if model_name == 'KNN Regression':
            clf = GridSearchCV(model, {'n_neighbors':[1, 3, 5], 'weights': ['uniform', 'distance']}, cv=3)
            X_train = scaler.fit_transform(X[train_i])
            clf.fit(X_train, y[train_i])
            #print(clf.best_params_)
            X_test = scaler.transform(X[test_i]) #normalize features
            preds = clf.predict(X_test)
            scores.append(mape(y[test_i], preds))
        else:
            model.fit(X[train_i], y[train_i])
            preds = model.predict(X[test_i])
            scores.append(mape(y[test_i], preds))
    scores = np.array(scores)
    table_rows.append((model_name, scores.mean(), scores.std(), scores.min(), scores.max()))
    with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
        line = 'Ship squares;Whole ship cable quantity;m;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        file_object.write(line)
display_table(table_rows)



Data array:  (11, 2)
Features (11, 1)


In [4]:
# Whole ship cable quantities
# Predictors:
# GT
# Ship squares
# Algorithms:
# Linear regression
# KNN regression


# Get the data
cursor.execute("SELECT gross_tonnage, ship_squares , sum(amount) FROM routed as r"
    " LEFT JOIN (SELECT project_id, sum(squares) as ship_squares FROM areas GROUP BY project_id) as a ON a.project_id=r.project_id"
    " LEFT JOIN projects as p ON p.project_id=r.project_id"
    " GROUP BY r.project_id, ship_squares, gross_tonnage")
data = np.array(cursor.fetchall()).astype('float32')
print('Data array: ', data.shape)
X = data[:, [0, 1]]
print('Features', X.shape)
y = data[:, -1]


# init regression model
models = dict()
models['Linear regression'] = linear_model.LinearRegression()
models['KNN Regression'] = KNeighborsRegressor()

scaler = pp.MinMaxScaler()

# train and evaluate all models with lpo
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
for model_name in models.keys():
    model = models[model_name]
    scores = []
    for train_i, test_i in lpo.split(X):
        if model_name == 'KNN Regression':
            clf = GridSearchCV(model, {'n_neighbors':[1, 3, 5], 'weights': ['uniform', 'distance']}, cv=3)
            X_train = scaler.fit_transform(X[train_i])
            clf.fit(X_train, y[train_i])
            #print(clf.best_params_)
            X_test = scaler.transform(X[test_i]) #normalize features
            preds = clf.predict(X_test)
            scores.append(mape(y[test_i], preds))
        else:
            model.fit(X[train_i], y[train_i])
            preds = model.predict(X[test_i])
            scores.append(mape(y[test_i], preds))
    scores = np.array(scores)
    table_rows.append((model_name, scores.mean(), scores.std(), scores.min(), scores.max()))
    with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
        line = 'GT, Ship squares;Whole ship cable quantity;m;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        file_object.write(line)
display_table(table_rows)


Data array:  (11, 3)
Features (11, 2)


In [5]:
# Whole ship cable quantities
# Predictors:
# Area type (and list of areas in the target ship)
# Algorithms:
# Mean

# Get the data
cursor.execute("SELECT DISTINCT project_id FROM routed")
data = np.array(cursor.fetchall()).astype('int32')
X = data[:, 0]
# init lpo split
p = 2 #ships
lpo = LeavePOut(p)
# train and evaluate model with lpo
scores = []
for train_i, test_i in lpo.split(X):
    s1 = X[test_i[0]]
    s2 = X[test_i[1]]
    cursor.execute(
        "SELECT predictions.cable_count as pred, sum(amount) as real_count FROM ("
        " SELECT project_id, sum(avg_count) as cable_count FROM areas LEFT JOIN ("
        " SELECT area_type, avg(area_cables) as avg_count FROM ("
        " SELECT project_id, area, sum(amount)/2 as area_cables FROM ("
        " SELECT project_id, start_area as area, sum(amount) as amount FROM routed"
        " WHERE project_id <> {} AND project_id <> {}"
        " GROUP BY start_area, project_id"
        " UNION"
        " SELECT project_id, end_area as area, sum(amount) as amount FROM routed"
        " WHERE project_id <> {} AND project_id <> {}"
        " GROUP BY end_area, project_id"
        " ) area_union GROUP BY project_id, area"
        " ) t_sums LEFT JOIN areas ON areas.project_id=t_sums.project_id AND areas.area=t_sums.area"
        " GROUP BY area_type"
        " ) avg_counts ON avg_counts.area_type=areas.area_type"
        " WHERE project_id={} OR project_id={}"
        " GROUP BY project_id"
        " ) predictions LEFT JOIN routed ON predictions.project_id=routed.project_id"
        " GROUP BY predictions.project_id, predictions.cable_count"
        .format(s1,s2,s1,s2,s1,s2)
    )
    data = np.array(cursor.fetchall())
    preds = data[:, 0]
    y_test = data[:, 1]
    scores.append(mape(y_test, preds))
    
scores = np.array(scores)
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
table_rows.append(('Mean', scores.mean(), scores.std(), scores.min(), scores.max()))
with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
    line = 'Area Type;Whole ship cable quantity;m;{};{};{};{};{}\n'.format('Mean', scores.mean(), scores.std(), scores.min(), scores.max())
    file_object.write(line)
display_table(table_rows)



In [6]:
# Whole ship cable quantities
# Predictors:
# Area squares
# Area type (and list of areas in the target ship)
# Algorithms:
# Linear regression

# Get the data

# get area types
cursor.execute("SELECT DISTINCT area_type FROM areas")
area_types = [item for t in cursor.fetchall() for item in t]

#get ship data
cursor.execute("SELECT project_id FROM routed GROUP BY project_id")
data = np.array(cursor.fetchall()).astype('int32')
ships = data[:, 0]

# init regression model
model = linear_model.LinearRegression()

# init lpo split
p = 2 #ships
lpo = LeavePOut(p)
# train and evaluate model with lpo
scores = []
for train_i, test_i in lpo.split(ships):
    # update area squares linear model coefficients and predictions
    cursor.execute("UPDATE areas SET pred_count = NULL")
    cursor.commit()
    s1 = ships[test_i[0]]
    s2 = ships[test_i[1]]
    for area_type in area_types:
        cursor.execute(
            " SELECT squares, cable_count FROM ("
            " SELECT area_union.project_id, area_union.area, area_type, squares, sum(amount)/2 as cable_count FROM ("
            " SELECT project_id, start_area as area, sum(amount) as amount FROM routed"
            " WHERE project_id <> {} AND project_id <> {}"
            " GROUP BY start_area, project_id"
            " UNION"
            " SELECT project_id, end_area as area, sum(amount) as amount FROM routed"
            " WHERE project_id <> {} AND project_id <> {}"
            " GROUP BY end_area, project_id"
            " ) area_union LEFT JOIN areas ON areas.project_id=area_union.project_id AND areas.area=area_union.area"
            " GROUP BY area_union.project_id, area_union.area, area_type, squares"
            " ) cable_counts WHERE area_type='{}' AND squares IS NOT NULL"
            .format(s1,s2,s1,s2,area_type)
        )
        data = np.array(cursor.fetchall()).astype('float32')
        if data.shape[0] == 0:
            continue
        X = data[:, 0].reshape((-1, 1))
        y = data[:, 1]
        model.fit(X, y)
        cursor.execute("UPDATE areas SET pred_count=squares*{} + {} WHERE area_type='{}'".format(model.coef_[0], model.intercept_, area_type))
        cursor.commit()
    # calculate ship wide predictions
    cursor.execute(
        "SELECT sum(pred_count) as preds, real_count FROM areas"
        " LEFT JOIN (SELECT project_id, sum(amount) as real_count FROM routed GROUP BY project_id) as real_counts ON real_counts.project_id=areas.project_id"
        " WHERE areas.project_id={} OR areas.project_id={} GROUP BY areas.project_id, real_count"
        .format(s1, s2)
        )
    data = np.array(cursor.fetchall()).astype('float32')
    preds = data[:, 0]
    y_test = data[:, 1]
    scores.append(mape(y_test, preds))

scores = np.array(scores)
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
table_rows.append(('Linear regression', scores.mean(), scores.std(), scores.min(), scores.max()))
with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
    line = 'Area squares, Area type;Whole ship cable quantity;m;{};{};{};{};{}\n'.format('Linear regression', scores.mean(), scores.std(), scores.min(), scores.max())
    file_object.write(line)
display_table(table_rows)



In [7]:
# Whole ship cable quantities
# Predictors:
# Area squares
# GT
# Area type (and list of areas in the target ship)
# Algorithms:
# Linear regression

# Get the data

# get area types
cursor.execute("SELECT DISTINCT area_type FROM areas")
area_types = [item for t in cursor.fetchall() for item in t]

#get ship data
cursor.execute("SELECT project_id FROM routed GROUP BY project_id")
data = np.array(cursor.fetchall()).astype('int32')
ships = data[:, 0]

# init regression model
model = linear_model.LinearRegression()


# init lpo split
p = 2 #ships
lpo = LeavePOut(p)
# train and evaluate model with lpo
scores = []
for train_i, test_i in lpo.split(ships):
    # update area squares linear model coefficients and predictions
    cursor.execute("UPDATE areas SET pred_count = NULL")
    cursor.commit()
    s1 = ships[test_i[0]]
    s2 = ships[test_i[1]]
    for area_type in area_types:
        cursor.execute(
            " SELECT gross_tonnage, squares, cable_count FROM ("
            " SELECT area_union.project_id as proj, area_union.area, area_type, squares, sum(amount)/2 as cable_count FROM ("
            " SELECT project_id, start_area as area, sum(amount) as amount FROM routed"
            " WHERE project_id <> {} AND project_id <> {}"
            " GROUP BY start_area, project_id"
            " UNION"
            " SELECT project_id, end_area as area, sum(amount) as amount FROM routed"
            " WHERE project_id <> {} AND project_id <> {}"
            " GROUP BY end_area, project_id"
            " ) area_union LEFT JOIN areas ON areas.project_id=area_union.project_id AND areas.area=area_union.area"
            " GROUP BY area_union.project_id, area_union.area, area_type, squares"
            " ) cable_counts LEFT JOIN projects ON projects.project_id=proj"
            " WHERE area_type='{}' AND squares IS NOT NULL"
            .format(s1,s2,s1,s2,area_type)
        )
        data = np.array(cursor.fetchall()).astype('float32')
        if data.shape[0] == 0:
            continue
        X = data[:, [0, 1]]
        y = data[:, -1]
        model.fit(X, y)
        cursor.execute(
            "UPDATE areas SET areas.pred_count=projects.gross_tonnage*{} + areas.squares*{} + {}"
            " FROM projects"
            " WHERE areas.project_id=projects.project_id AND areas.area_type='{}'"
            .format(model.coef_[0], model.coef_[1], model.intercept_, area_type)
            )
        cursor.commit()
    # calculate ship wide predictions
    cursor.execute(
        "SELECT sum(pred_count) as preds, real_count FROM areas"
        " LEFT JOIN (SELECT project_id, sum(amount) as real_count FROM routed GROUP BY project_id) as real_counts ON real_counts.project_id=areas.project_id"
        " WHERE areas.project_id={} OR areas.project_id={} GROUP BY areas.project_id, real_count"
        .format(s1, s2)
        )
    data = np.array(cursor.fetchall()).astype('float32')
    preds = data[:, 0]
    y_test = data[:, 1]
    scores.append(mape(y_test, preds))

scores = np.array(scores)
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
table_rows.append(('Linear regression', scores.mean(), scores.std(), scores.min(), scores.max()))
with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
    line = 'GT, Area squares, Area type;Whole ship cable quantity;m;{};{};{};{};{}\n'.format('Linear regression', scores.mean(), scores.std(), scores.min(), scores.max())
    file_object.write(line)
display_table(table_rows)



In [9]:
# Whole ship cable quantities
# Predictors:
# Area squares
# Area type (and list of areas in the target ships)
# Algorithms:
# K-NN Regression

# Get the data

# get area types
cursor.execute("SELECT DISTINCT area_type FROM areas")
area_types = [item for t in cursor.fetchall() for item in t]

#get ship data
cursor.execute("SELECT project_id, sum(amount) FROM routed GROUP BY project_id")
data = np.array(cursor.fetchall()).astype('int32')
ships = data[:, 0]
values = data[:, 1]
zips = zip(ships, values)
cable_counts = dict(zips)

# init regression models
models = dict()
models['KNN Regression'] = KNeighborsRegressor()


# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

# train and evaluate models with lpo
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
for model_name in models.keys():
    model = models[model_name]
    scores = []
    for train_i, test_i in lpo.split(ships):
        # update area squares for training data
        cursor.execute("UPDATE areas SET pred_count = NULL")
        cursor.commit()
        s1 = ships[test_i[0]]
        s2 = ships[test_i[1]]
        pred_counts = {s1:0, s2:0}
        for area_type in area_types:
            cursor.execute(
                " SELECT squares, cable_count FROM ("
                " SELECT area_union.project_id, area_union.area, area_type, squares, sum(amount)/2 as cable_count FROM ("
                " SELECT project_id, start_area as area, sum(amount) as amount FROM routed"
                " WHERE project_id <> {} AND project_id <> {}"
                " GROUP BY start_area, project_id"
                " UNION"
                " SELECT project_id, end_area as area, sum(amount) as amount FROM routed"
                " WHERE project_id <> {} AND project_id <> {}"
                " GROUP BY end_area, project_id"
                " ) area_union LEFT JOIN areas ON areas.project_id=area_union.project_id AND areas.area=area_union.area"
                " GROUP BY area_union.project_id, area_union.area, area_type, squares"
                " ) cable_counts WHERE area_type='{}' AND squares IS NOT NULL"
                .format(s1,s2,s1,s2,area_type)
            )
            data = np.array(cursor.fetchall()).astype('float32')
            if data.shape[0] == 0:
                continue
            X_train = data[:, 0].reshape((-1, 1))
            y_train = data[:, 1]

            # normalize features 
            scaler = pp.MinMaxScaler()
            X_train = scaler.fit_transform(X_train)
            if X_train.shape[0] < 7:
                continue
            elif X_train.shape[0] < 10:
                cv = LeavePOut(p)
            else:
                cv = 3
            clf = GridSearchCV(model, {'n_neighbors':[1, 3, 5], 'weights': ['uniform', 'distance']}, cv=cv)
            clf.fit(X_train, y_train)
            # calculate ship predictions
            for ship in [s1, s2]:
                cursor.execute("SELECT squares FROM areas WHERE area_type='{}' AND project_id={} AND squares IS NOT NULL".format(area_type, ship))
                data = np.array(cursor.fetchall()).astype('float32')
                if data.shape[0] < 1: #check if enough samples
                    continue
                X_test = data[:, 0].reshape(-1, 1)
                X_test = scaler.transform(X_test) #normalize features
                preds = clf.predict(X_test)
                pred_counts[ship] += preds.sum()

        # calculate ship wide predictions and cv score
        y_test = []
        preds = []
        for ship in [s1, s2]:
            y_test.append(cable_counts[ship])
            preds.append(pred_counts[ship])
        score = mape(y_test, preds)
        scores.append(score)

    scores = np.array(scores)
    table_rows.append((model_name, scores.mean(), scores.std(), scores.min(), scores.max()))
    with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
        line = 'Area squares, Area type;Whole ship cable quantity;m;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        file_object.write(line)
display_table(table_rows)



In [10]:
# Whole ship cable quantities
# Predictors:
# GT
# Area squares
# Area type (and list of areas in the target ship)
# Algorithms:
# K-NN Regression

# Get the data

# get area types
cursor.execute("SELECT DISTINCT area_type FROM areas")
area_types = [item for t in cursor.fetchall() for item in t]

#get ship data
cursor.execute("SELECT project_id, sum(amount) FROM routed GROUP BY project_id")
data = np.array(cursor.fetchall()).astype('int32')
ships = data[:, 0]
values = data[:, 1]
zips = zip(ships, values)
cable_counts = dict(zips)

# init regression models
models = dict()
models['KNN Regression'] = KNeighborsRegressor()

# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

# train and evaluate models with lpo
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
for model_name in models.keys():
    model = models[model_name]
    scores = []
    for train_i, test_i in lpo.split(ships):
        # update area squares for training data
        cursor.execute("UPDATE areas SET pred_count = NULL")
        cursor.commit()
        s1 = ships[test_i[0]]
        s2 = ships[test_i[1]]
        pred_counts = {s1:0, s2:0}
        for area_type in area_types:
            cursor.execute(
                " SELECT gross_tonnage, squares, cable_count FROM ("
                " SELECT area_union.project_id, area_union.area, area_type, squares, sum(amount)/2 as cable_count FROM ("
                " SELECT project_id, start_area as area, sum(amount) as amount FROM routed"
                " WHERE project_id <> {} AND project_id <> {}"
                " GROUP BY start_area, project_id"
                " UNION"
                " SELECT project_id, end_area as area, sum(amount) as amount FROM routed"
                " WHERE project_id <> {} AND project_id <> {}"
                " GROUP BY end_area, project_id"
                " ) area_union LEFT JOIN areas ON areas.project_id=area_union.project_id AND areas.area=area_union.area"
                " GROUP BY area_union.project_id, area_union.area, area_type, squares"
                " ) cable_counts "
                " LEFT JOIN projects ON projects.project_id=cable_counts.project_id"
                " WHERE area_type='{}' AND squares IS NOT NULL"
                .format(s1,s2,s1,s2,area_type)
            )
            data = np.array(cursor.fetchall()).astype('float32')
            if data.shape[0] == 0:
                continue
            X_train = data[:, [0, 1]]
            y_train = data[:, -1]

            # normalize features
            scaler = pp.MinMaxScaler()
            X_train = scaler.fit_transform(X_train)

            if X_train.shape[0] < 7:
                continue
            elif X_train.shape[0] < 10:
                cv = LeavePOut(p)
            else:
                cv = 3
            clf = GridSearchCV(model, {'n_neighbors':[1, 3, 5], 'weights': ['uniform', 'distance']}, cv=cv)
            clf.fit(X_train, y_train)

            # calculate ship predictions
            for ship in [s1, s2]:
                cursor.execute(
                    "SELECT gross_tonnage, squares FROM areas "
                    " LEFT JOIN projects ON projects.project_id=areas.project_id"
                    " WHERE area_type='{}' AND areas.project_id={} AND squares IS NOT NULL"
                    .format(area_type, ship)
                    )
                data = np.array(cursor.fetchall()).astype('float32')
                if data.shape[0] < 1: #check if enough samples
                    continue
                X_test = data[:, [0, 1]]
                X_test = scaler.transform(X_test) #normalize features
                preds = clf.predict(X_test)
                pred_counts[ship] += preds.sum()

        # calculate ship wide predictions and cv score
        y_test = []
        preds = []
        for ship in [s1, s2]:
            y_test.append(cable_counts[ship])
            preds.append(pred_counts[ship])
        score = mape(y_test, preds)
        scores.append(score)

    scores = np.array(scores)
    table_rows.append((model_name, scores.mean(), scores.std(), scores.min(), scores.max()))
    with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
        line = 'GT, Area squares, Area type;Whole ship cable quantity;m;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        file_object.write(line)
display_table(table_rows)



In [11]:
# Whole ship cable quantities (pcs, for comparison)
# Predictors:
# GT
# Algorithms:
# Linear regression

# Get the data
cursor.execute("SELECT gross_tonnage, count(*) FROM routed as r"
    " LEFT JOIN projects as p ON p.project_id=r.project_id"
    " GROUP BY r.project_id, gross_tonnage")
data = np.array(cursor.fetchall()).astype('float32')
print('Data array: ', data.shape)
X = data[:, 0].reshape((-1, 1))
print('Features', X.shape)
y = data[:, -1]


# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

# init regression model
models = dict()
models['Linear regression'] = linear_model.LinearRegression()

# train and evaluate all models with lpo
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
for model_name in models.keys():
    model = models[model_name]
    scores = []
    for train_i, test_i in lpo.split(X):
        model.fit(X[train_i], y[train_i])
        preds = model.predict(X[test_i])
        scores.append(mape(y[test_i], preds))
    scores = np.array(scores)
    table_rows.append((model_name, scores.mean(), scores.std(), scores.min(), scores.max()))
    with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
        line = 'GT;Whole ship cable quantity (for comparison);pcs;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        file_object.write(line)
display_table(table_rows)


Data array:  (11, 2)
Features (11, 1)


In [12]:
# Whole ship cable quantities (pcs, for comparison)
# Predictors:
# Area squares
# Area type (and list of areas in the target ship)
# Algorithms:
# K-NN Regression

# Get the data

# get area types
cursor.execute("SELECT DISTINCT area_type FROM areas")
area_types = [item for t in cursor.fetchall() for item in t]

#get ship data
cursor.execute("SELECT project_id, count(*) FROM routed GROUP BY project_id")
data = np.array(cursor.fetchall()).astype('int32')
ships = data[:, 0]
values = data[:, 1]
zips = zip(ships, values)
cable_counts = dict(zips)

# init regression models
models = dict()
models['KNN Regression'] = KNeighborsRegressor()

# init lpo split
p = 2 #ships
lpo = LeavePOut(p)

# train and evaluate models with lpo
table_rows = [['Model', 'mean', 'std', 'min', 'max']]
for model_name in models.keys():
    model = models[model_name]
    scores = []
    for train_i, test_i in lpo.split(ships):
        # update area squares for training data
        cursor.execute("UPDATE areas SET pred_count = NULL")
        cursor.commit()
        s1 = ships[test_i[0]]
        s2 = ships[test_i[1]]
        pred_counts = {s1:0, s2:0}
        for area_type in area_types:
            cursor.execute(
                " SELECT squares, cable_count FROM ("
                " SELECT area_union.project_id, area_union.area, area_type, squares, sum(amount)/2 as cable_count FROM ("
                " SELECT project_id, start_area as area, count(*) as amount FROM routed"
                " WHERE project_id <> {} AND project_id <> {}"
                " GROUP BY start_area, project_id"
                " UNION"
                " SELECT project_id, end_area as area, count(*) as amount FROM routed"
                " WHERE project_id <> {} AND project_id <> {}"
                " GROUP BY end_area, project_id"
                " ) area_union LEFT JOIN areas ON areas.project_id=area_union.project_id AND areas.area=area_union.area"
                " GROUP BY area_union.project_id, area_union.area, area_type, squares"
                " ) cable_counts WHERE area_type='{}' AND squares IS NOT NULL"
                .format(s1,s2,s1,s2,area_type)
            )
            data = np.array(cursor.fetchall()).astype('float32')
            if data.shape[0] == 0:
                continue
            X_train = data[:, 0].reshape((-1, 1))
            y_train = data[:, 1]

            # normalize features
            scaler = pp.MinMaxScaler()
            X_train = scaler.fit_transform(X_train)

            if X_train.shape[0] < 7:
                continue
            elif X_train.shape[0] < 10:
                cv = LeavePOut(p)
            else:
                cv = 3
            clf = GridSearchCV(model, {'n_neighbors':[1, 3, 5], 'weights': ['uniform', 'distance']}, cv=cv)
            clf.fit(X_train, y_train)

            # calculate ship predictions
            for ship in [s1, s2]:
                cursor.execute("SELECT squares FROM areas WHERE area_type='{}' AND project_id={} AND squares IS NOT NULL".format(area_type, ship))
                data = np.array(cursor.fetchall()).astype('float32')
                if data.shape[0] < 1: #check if enough samples
                    continue
                X_test = data[:, 0].reshape(-1, 1)
                X_test = scaler.transform(X_test) #normalize features
                preds = clf.predict(X_test)
                pred_counts[ship] += preds.sum()

        # calculate ship wide predictions and cv score
        y_test = []
        preds = []
        for ship in [s1, s2]:
            y_test.append(cable_counts[ship])
            preds.append(pred_counts[ship])
        score = mape(y_test, preds)
        scores.append(score)

    scores = np.array(scores)
    table_rows.append((model_name, scores.mean(), scores.std(), scores.min(), scores.max()))
    with open("static_whole_ship_cable_quantities.txt", "a") as file_object:
        line = 'Area squares, Area type;Whole ship cable quantity (for comparison);pcs;{};{};{};{};{}\n'.format(model_name, scores.mean(), scores.std(), scores.min(), scores.max())
        file_object.write(line)
display_table(table_rows)
